<a href="https://colab.research.google.com/github/BalchandraJha/Pneumonia_Prediction_using_Chest_Xrays/blob/master/Pneumonia_Prediction_Chest_Xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf

In [3]:
from keras.layers import Dense,Flatten,Lambda,Input
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [5]:
# resize all the image
IMAGE_SIZE = [224,224]

In [4]:
train_path = '/content/drive/My Drive/Colab Notebooks/DATASET/train'
valid_path = '/content/drive/My Drive/Colab Notebooks/DATASET/test'

In [6]:
vgg = VGG16(input_shape = IMAGE_SIZE+[3],weights = 'imagenet',include_top =False)

58892288/58889256 [==============================] - 1s 0us/step


In [7]:
for layer in vgg.layers:
  layer.trainable = False

In [8]:
folder = glob('/content/drive/My Drive/Colab Notebooks/DATASET/train/*')

In [9]:
folder

['/content/drive/My Drive/Colab Notebooks/DATASET/train/NORMAL',
 '/content/drive/My Drive/Colab Notebooks/DATASET/train/PNEUMONIA']

In [10]:
x = Flatten()(vgg.output)

In [11]:
prediction = Dense(len(folder),activation= 'softmax')(x)

In [12]:
model = Model(inputs= vgg.inputs,outputs = prediction)

In [13]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [14]:
model.compile(loss= 'categorical_crossentropy',optimizer= 'adam',metrics= ['accuracy'])

In [15]:
from keras.preprocessing.image import ImageDataGenerator


In [16]:
train_datagen = ImageDataGenerator(rescale= 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [17]:
test_datagen = ImageDataGenerator(rescale= 1./255)

In [18]:
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/DATASET/train',
                                                 target_size =(224,224),batch_size = 32,class_mode = 'categorical')

Found 870 images belonging to 2 classes.


In [19]:
test_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/DATASET/test',
                                                 target_size =(224,224),batch_size = 32,class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [20]:
r = model.fit_generator(training_set,validation_data= test_set,epochs= 5, steps_per_epoch=len(training_set),validation_steps=len(test_set))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
28/28 [==============================] - 962s 34s/step - loss: 0.3786 - accuracy: 0.8149 - val_loss: 0.3162 - val_accuracy: 0.8750
Epoch 2/5
28/28 [==============================] - 38s 1s/step - loss: 0.1444 - accuracy: 0.9471 - val_loss: 0.3171 - val_accuracy: 0.8862
Epoch 3/5
28/28 [==============================] - 38s 1s/step - loss: 0.1388 - accuracy: 0.9460 - val_loss: 0.2349 - val_accuracy: 0.9119
Epoch 4/5
28/28 [==============================] - 39s 1s/step - loss: 0.1373 - accuracy: 0.9448 - val_loss: 0.2469 - val_accuracy: 0.9054
Epoch 5/5
28/28 [==============================] - 38s 1s/step - loss: 0.1074 - accuracy: 0.9586 - val_loss: 0.3592 - val_accuracy: 0.8926


In [21]:
from keras.models import load_model
model.save('model_vgg16.h5')

In [22]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = load_model('model_vgg16.h5')
img = image.load_img('/content/drive/My Drive/Colab Notebooks/DATASET/val/NORMAL/NORMAL2-IM-1427-0001.jpeg',target_size = (224,224))
x= image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
img_data = preprocess_input(x)
classes = model.predict(img_data)

In [23]:
classes

array([[1., 0.]], dtype=float32)